CNN med Pytorch

In [33]:
import numpy as np
import matplotlib.pyplot as plt
# import pandas as pd

#other libraries
# from tqdm import tqdm
# import time
# import random

import sys
from pathlib import Path


#torch specific
import torch
import torchvision as torchv
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch import Tensor
from torch.utils import data

from sklearn.model_selection import train_test_split
import wandb

from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn

In [34]:
module_path = str(Path.cwd().parents[0].parents[0] / "methods")

if module_path not in sys.path:
    sys.path.append(module_path)

from dataloader import *
# from plotCreator import *

data_path0 = str(Path.cwd().parents[0].parents[0] / "data" / "BH_n4_M10_res50_15000_events.h5")
data_path1 = str(Path.cwd().parents[0].parents[0] / "data" / "PP13-Sphaleron-THR9-FRZ15-NB0-NSUBPALL_res50_15000_events.h5")

In [35]:
bhArray = dataToArray(data_path0)
sphArray = dataToArray(data_path1)

# print(bhArray.shape)
# print(sphArray.shape)

In [36]:
#Kombinerer dataene for å kunne kjøre gjennom modellen på et samlet datasett
dataArray = np.concatenate((bhArray,sphArray),axis=0)

# np.shape(dataArray)

In [37]:
# Labeler tabelle med 1 og 0 (0 = svart hull, 1 = spahleron)
labelsArray = np.concatenate((np.zeros(np.shape(bhArray)[0]),np.ones(np.shape(sphArray)[0])),axis=0)

Sjekk om køyrer på GPU eller CPU

In [38]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [39]:
#Split data 75% i train og 25% i test
trainData, testData, trainLabels, testLabels = train_test_split(dataArray, labelsArray, random_state=42)

In [40]:
trainData = torch.from_numpy(trainData)
testData = torch.from_numpy(testData)
trainLabels = torch.from_numpy(trainLabels)
testLabels = torch.from_numpy(testLabels)

In [41]:
train = torch.utils.data.TensorDataset(trainData, trainLabels)
test = torch.utils.data.TensorDataset(testData, testLabels)

In [42]:
trainLoader = DataLoader(train, shuffle=True, batch_size=50)
testLoader = DataLoader(test, shuffle=True, batch_size=50)

Her er vårt CNN modell

Her skal det komme endrigane for å tilpasse første CNN

In [66]:
# Her kommer modellen inn
class ConvModel(nn.Module):
    def __init__(self, dropout):

        super(ConvModel, self).__init__()
        # 2 cov lag blir opprette. Bildene har x, y og z verdi. 
        
        # Define the convolutional layers with rotational and reflectional symmetry filters
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, padding=0)
        self.conv1_filter = nn.Parameter(torch.ones((16, 3, 3, 3)))
        self.conv1_filter.detach_()
        
        self.conv1_filter[:, :, 1, 1] = 0  # set the center of the filter to zero
        self.conv1_filter[:, :, :, 1] = self.conv1_filter.flip(dims=[3])  # set the right half of the filter
        self.conv1_filter[:, :, 0, 0] = self.conv1_filter[:, :, 2, 2]  # set the top-left and bottom-right corners
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=64, kernel_size=3, padding=0)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=256, kernel_size=3, padding=0)

        self.fc1 = nn.Linear(3*3*256, 128)
        self.fc2 = nn.Linear(128,2)

        self.dropout = nn.Dropout(dropout)


    #FIX THIS ASSHOLE
    def forward(self, x:Tensor):
        x = F.conv2d(x, self.conv1_filter, bias=self.conv1.bias, stride=1, padding=1)
        x = F.relu(x)
        x = F.max_pool2d(x,2)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x,2)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x,3)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
 
        return x

summere modellen sine parametera

In [67]:
from torchsummary import summary

model = ConvModel(0).to(device)
summary(model, (3, 50, 50))

RuntimeError: expand(torch.FloatTensor{[16, 3, 3, 3]}, size=[16, 3, 3]): the number of sizes provided (3) must be greater or equal to the number of dimensions in the tensor (4)

Trene modellen vår

In [68]:

def trainFunction(config=None):

        #initialize variables
        model = ConvModel(config.dropout).to(device)

        #data
        trainLoader = DataLoader(train, shuffle=True, batch_size=config.batch_size)
        testLoader = DataLoader(test, shuffle=True, batch_size=config.batch_size)
        #data end
        
        
        #Her komemr data augmetnation?
        
        
        #START
        for epoch in range(20):

            #training variables
  
            y_pred = []
            y_true = []
            
            #training loop
            model.train()
            for i, data in enumerate(trainLoader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                output = model(inputs.permute(0,3,1,2)) # change 
                output1 = (torch.max(output.to(device), 1)[1]) #predicted output
                
                y_pred.extend(output1) # Save Prediction
                
                y_true.extend(labels) # Save Truth
           
            #test variables
            y_pred = []
            y_true = []
            
            #test loop
            model.eval()
            for j, data in enumerate(testLoader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                
                output = model(inputs.permute(0,3,1,2))# Feed Network
                output1 = (torch.max(output.to(device), 1)[1])
                y_pred.extend(output1) # Save Prediction
                
                y_true.extend(labels) # Save Truth
            #test loop end
            
            # #Here comes print
            
        #END

kanskje lurt å se på wandb for å deploye modellen 